# Transform MGNify sample data into ML compatible input tables

## Shotgun Taxonomy SSU tables

* We filter out shotgun sequences only
* Rows are each sample
* Each sample has a descriptor of its environment
* Each sample has a column entry for *every* taxon across all samples
* Each taxa column has an abundance value for that sample
* Input data consists of all taxa columns 
* Output consists of the environment column

In [3]:
import os
import json
import glob
import csv
import pandas as pd

base_dir = 'data/mgnify/studies'
out_path = 'data/mgnify/GO_aggregated.tsv'

def iter_studies():
    for dir_name in os.listdir(base_dir):
        dir_path = os.path.join(base_dir, dir_name)
        if not os.path.isdir(dir_path):
            continue
        study_json_path = os.path.join(dir_path, 'study.json')
        with open(study_json_path) as fd:
            study_json = json.load(fd)
        yield dir_path, study_json

all_go_terms = set()
for study_path, study_json in iter_studies():
    study_id = study_json['id']
    for tsv_path in glob.iglob(os.path.join(study_path, "*GO*.tsv")):
        with open(tsv_path) as fd:
            tsv_df = pd.read_csv(fd, sep='\t')
            go_terms = set(tsv_df['GO'])
            all_go_terms.update(go_terms)
        ids = tsv_df.keys()[3:]

# print('all_go_terms', all_go_terms)
go_terms_ordered = list(all_go_terms)
out_data = pd.DataFrame(columns=['id', 'study_id'] + go_terms_ordered)

In [9]:
out_data

,id,study_id,GO:0043130,GO:0055074,GO:0055117,GO:0046933,GO:0006302,GO:0008643,GO:0043752,GO:0007026,...,GO:0019357,GO:0006527,GO:0004114,GO:0046423,GO:0034194,GO:0032183,GO:0007618,GO:0030097,GO:0004520,GO:0033739


In [10]:
# Stream data to a TSV file

with open('go_aggregated.tsv', 'w', newline='') as fd:
    writer = csv.writer(fd, delimiter='\t')
    # Write headers
    headers = ['id', 'study_id'] + go_terms_ordered
    writer.writerow(headers)
    for study_path, study_json in iter_studies():
        study_id = study_json['id']
        for tsv_path in glob.iglob(os.path.join(study_path, "*GO*.tsv")):
            print('Processing', tsv_path)
            with open(tsv_path) as fd:
                tsv_df = pd.read_csv(fd, sep='\t')
            rows = dict()
            ids = tsv_df.keys()[3:]
            for idx, row in tsv_df.iterrows():
                go_term = row['GO']
                for _id in ids:
                    if _id not in rows:
                        rows[_id] = {'id': _id, 'study_id': study_id}
                    rows[_id][go_term] = row[_id]
            for (_, row) in rows.items():
                row_flat = [row['id'], row['study_id']]
                for go_term in go_terms_ordered:
                    row_flat.append(rows[_id].get(go_term, 0))
                writer.writerow(row_flat)
print('Done!')

Processing data/mgnify/studies/MGYS00002076/SRP076308_GO_abundances_v4.0.tsv
Processing data/mgnify/studies/MGYS00000323/ERP001979_GO_abundances_v1.0.tsv
Processing data/mgnify/studies/MGYS00003358/ERP108758_GO_abundances_v4.1.tsv
Processing data/mgnify/studies/MGYS00002028/ERP104187_GO_abundances_v4.0.tsv
Processing data/mgnify/studies/MGYS00002028/ERP104187_GO_abundances_v4.1.tsv
Processing data/mgnify/studies/MGYS00001568/ERP004475_GO_abundances_v3.0.tsv
Processing data/mgnify/studies/MGYS00004737/ERP111034_GO_abundances_v4.1.tsv
Processing data/mgnify/studies/MGYS00000531/ERP011824_GO_abundances_v2.0.tsv
Processing data/mgnify/studies/MGYS00003390/ERP111166_GO_abundances_v4.1.tsv
Processing data/mgnify/studies/MGYS00005065/ERP105157_GO_abundances_v4.1.tsv
Processing data/mgnify/studies/MGYS00004721/ERP114670_GO_abundances_v4.1.tsv
Processing data/mgnify/studies/MGYS00001318/ERP016770_GO_abundances_v3.0.tsv
Processing data/mgnify/studies/MGYS00005128/ERP117839_GO_abundances_v4.1.tsv

In [7]:
with open('go_aggregated.tsv') as fd:
    df = pd.read_csv(fd, sep='\t')
df.head()

,id,study_id,GO:0043130,GO:0055074,GO:0055117,GO:0046933,GO:0006302,GO:0008643,GO:0043752,GO:0007026,...,GO:0019357,GO:0006527,GO:0004114,GO:0046423,GO:0034194,GO:0032183,GO:0007618,GO:0030097,GO:0004520,GO:0033739
0,SRR3650980,MGYS00002076,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SRR3650981,MGYS00002076,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SRR3650982,MGYS00002076,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SRR3650983,MGYS00002076,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SRR3650984,MGYS00002076,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# TODO

* Confirm that the GO terms line up with the correct abundances for the samples in go_aggregated.tsv
* Take a subset of go_aggregated.tsv with sample IDs and add metadata

In [8]:
# Take a subset of go_aggregated.tsv with sample IDs

sampleid_subset = df.loc[df['id'].str.startswith('SRS')]
sampleid_subset

,id,study_id,GO:0043130,GO:0055074,GO:0055117,GO:0046933,GO:0006302,GO:0008643,GO:0043752,GO:0007026,...,GO:0019357,GO:0006527,GO:0004114,GO:0046423,GO:0034194,GO:0032183,GO:0007618,GO:0030097,GO:0004520,GO:0033739
1399,SRS000998_G,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1400,SRS000999_G,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1401,SRS001000_G,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1402,SRS001001_G,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1403,SRS001002_G,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61860,SRS070535_G,MGYS00000277,1,0,0,600,1,30,27,0,...,0,29,0,0,0,0,0,0,33,0
61861,SRS070536_G,MGYS00000277,1,0,0,600,1,30,27,0,...,0,29,0,0,0,0,0,0,33,0
63163,SRS009922_G,MGYS00000256,0,0,0,217,0,0,9,0,...,0,56,4,0,0,0,0,0,40,0
67789,SRS004796_G,MGYS00000288,0,0,0,51,5,0,37,0,...,0,21,0,0,0,0,0,0,2,0


In [9]:
# 241 sample IDs have an extraneous "_G" suffix
print(len(sampleid_subset.loc[sampleid_subset['id'].str.endswith('_G')]))

# 10 sample IDs have an extraneous "_G" suffix
print(len(sampleid_subset.loc[sampleid_subset['id'].str.endswith('_T')]))

241
10


In [10]:
sampleid_subset1 = sampleid_subset.replace(to_replace=r'_(G|T)$', value='', regex=True)

In [11]:
# 241 sample IDs have an extraneous "_G" suffix
print(len(sampleid_subset1.loc[sampleid_subset1['id'].str.endswith('_G')]))

# 10 sample IDs have an extraneous "_G" suffix
print(len(sampleid_subset1.loc[sampleid_subset1['id'].str.endswith('_T')]))

0
0


In [12]:
sampleid_subset1

,id,study_id,GO:0043130,GO:0055074,GO:0055117,GO:0046933,GO:0006302,GO:0008643,GO:0043752,GO:0007026,...,GO:0019357,GO:0006527,GO:0004114,GO:0046423,GO:0034194,GO:0032183,GO:0007618,GO:0030097,GO:0004520,GO:0033739
1399,SRS000998,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1400,SRS000999,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1401,SRS001000,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1402,SRS001001,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
1403,SRS001002,MGYS00000258,1,0,0,184,0,8,18,0,...,0,14,0,0,0,0,0,0,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61860,SRS070535,MGYS00000277,1,0,0,600,1,30,27,0,...,0,29,0,0,0,0,0,0,33,0
61861,SRS070536,MGYS00000277,1,0,0,600,1,30,27,0,...,0,29,0,0,0,0,0,0,33,0
63163,SRS009922,MGYS00000256,0,0,0,217,0,0,9,0,...,0,56,4,0,0,0,0,0,40,0
67789,SRS004796,MGYS00000288,0,0,0,51,5,0,37,0,...,0,21,0,0,0,0,0,0,2,0


In [14]:
# For each row in sampleid_subset1:
#   Open the study directory
#   Parse the samples.json file
#   Find the current sample within the data array
# Write all the data to a TSV

with open('go_5.0_biomes.tsv', 'w', newline='') as fd:
    writer = csv.writer(fd, delimiter='\t')
    # Write headers
    headers = ['id', 'study_id', 'biome', 'exptype'] + go_terms_ordered
    writer.writerow(headers) 
    for (idx, row) in sampleid_subset1.iterrows():
        study_id = row['study_id']
        sample_id = row['id']
        # Fetch the biome
        with open(os.path.join('data/mgnify/studies', study_id, 'samples.json')) as fd:
            samples_json = json.load(fd)
        biome = None
        for sample in samples_json['data']:
            if sample['id'] != sample_id:
                continue
            biome = sample['relationships']['biome']['data']['id']
        # Fetch the experiment type
        with open(os.path.join('data/mgnify/studies', study_id, 'analyses.json')) as fd:
            analyses_json = json.load(fd)
        exptype = 'undefined'
        for analysis in analyses_json['data']:
            an_sample_id = analysis['relationships']['sample']['data']['id']
            if an_sample_id != sample_id:
                continue
            exptype = analysis['attributes']['experiment-type']
        if biome is not None:
            row = [sample_id, study_id, biome, exptype] + list(row[2:])
            writer.writerow(row)
        print('Wrote', sample_id)
print('Done!')

Wrote SRS000998
Wrote SRS000999
Wrote SRS001000
Wrote SRS001001
Wrote SRS001002
Wrote SRS001003
Wrote SRS001004
Wrote SRS001005
Wrote SRS001006
Wrote SRS001007
Wrote SRS001008
Wrote SRS001009
Wrote SRS001012
Wrote SRS001013
Wrote SRS001014
Wrote SRS001015
Wrote SRS085651
Wrote SRS085660
Wrote SRS085661
Wrote SRS085662
Wrote SRS085663
Wrote SRS085664
Wrote SRS085665
Wrote SRS085666
Wrote SRS085667
Wrote SRS085668
Wrote SRS085669
Wrote SRS085670
Wrote SRS085671
Wrote SRS085672
Wrote SRS085673
Wrote SRS085674
Wrote SRS085675
Wrote SRS085676
Wrote SRS085677
Wrote SRS085678
Wrote SRS085679
Wrote SRS085680
Wrote SRS085681
Wrote SRS085682
Wrote SRS085683
Wrote SRS085684
Wrote SRS085685
Wrote SRS085686
Wrote SRS085687
Wrote SRS085688
Wrote SRS085689
Wrote SRS085690
Wrote SRS085691
Wrote SRS085692
Wrote SRS085693
Wrote SRS085694
Wrote SRS085695
Wrote SRS085696
Wrote SRS085697
Wrote SRS085698
Wrote SRS085699
Wrote SRS085700
Wrote SRS085701
Wrote SRS085702
Wrote SRS085703
Wrote SRS085704
Wrote SR

In [79]:
with open('go_5.0_biomes.tsv') as fd:
    df = pandas.read_csv(fd, sep='\t')
df.head()

,id,study_id,biome,GO:0043130,GO:0055074,GO:0055117,GO:0046933,GO:0006302,GO:0008643,GO:0043752,...,GO:0019357,GO:0006527,GO:0004114,GO:0046423,GO:0034194,GO:0032183,GO:0007618,GO:0030097,GO:0004520,GO:0033739
0,SRS000998,MGYS00000258,root:Host-associated:Human:Digestive system:La...,1,0,0,184,0,8,18,...,0,14,0,0,0,0,0,0,30,0
1,SRS000999,MGYS00000258,root:Host-associated:Human:Digestive system:La...,1,0,0,184,0,8,18,...,0,14,0,0,0,0,0,0,30,0
2,SRS001000,MGYS00000258,root:Host-associated:Human:Digestive system:La...,1,0,0,184,0,8,18,...,0,14,0,0,0,0,0,0,30,0
3,SRS001001,MGYS00000258,root:Host-associated:Human:Digestive system:La...,1,0,0,184,0,8,18,...,0,14,0,0,0,0,0,0,30,0
4,SRS001002,MGYS00000258,root:Host-associated:Human:Digestive system:La...,1,0,0,184,0,8,18,...,0,14,0,0,0,0,0,0,30,0
